# 9. 策略梯度算法

## 9.1 简介
- 本书之前介绍的 Q-learning、DQN 及 DQN 改进算法都是基于价值（value-based）的方法，其中 Q-learning 是处理有限状态的算法，而 DQN 可以用来解决连续状态的问题。在强化学习中，除了基于值函数的方法，还有一支非常经典的方法，那就是基于策略（policy-based）的方法。对比两者，基于值函数的方法主要是学习值函数，然后根据值函数导出一个策略，学习过程中并不存在一个显式的策略；而基于策略的方法则是直接显式地学习一个目标策略。策略梯度是基于策略的方法的基础，本章从策略梯度算法说起。

## 9.2 策略梯度
- 基于策略的方法首先需要将策略参数化。假设目标策略 $\pi_{\theta}$ 是一个随机性策略，并且处处可微，其中 $\theta$ 是对应的参数。我们可以用一个线性模型或者神经网络模型来为这样一个策略函数建模，输入某个状态，然后输出一个动作的概率分布。我们的目标是要寻找一个最优策略并最大化这个策略在环境中的期望回报。我们将策略学习的目标函数定义为：

$$
J(\theta) = \mathbb{E}_{s_0}[V^{\pi_{\theta}}(s_0)]
$$

- 其中，$s_0$ 是初始状态。现在有了目标函数，我们将目标函数对策略 $\theta$ 求导，得到导数后，就可以用梯度上升的方法来最大化这个目标函数，从而得到最优策略。

- 第3章讲解过策略 $\pi$ 下的状态访问分布，在此用 $\nu$ 表示，然后我们对目标函数求梯度，可以得到如下式子，更详细的推导过程将在 9.6 节给出。

$$
\begin{aligned}
\nabla_{\theta}J(\theta) & \propto\sum_{s\in S}\nu^{\pi_{\theta}}(s)\sum_{a\in A}Q^{\pi_{\theta}}(s,a)\nabla_{\theta}\pi_{\theta}(a|s) \\
 & =\sum_{s\in S}\nu^{\pi_{\theta}}(s)\sum_{a\in A}\pi_{\theta}(a|s)Q^{\pi_{\theta}}(s,a)\frac{\nabla_{\theta}\pi_{\theta}(a|s)}{\pi_{\theta}(a|s)} \\
 & =\mathbb{E}_{\pi_\theta}[Q^{\pi_\theta}(s,a)\nabla_\theta\log\pi_\theta(a|s)]
\end{aligned}
$$

- 这个梯度可以用来更新策略。需要注意的是，因为上式中期望 $\mathbb{E}$ 的下标是 $\pi_{\theta}$，所以策略梯度算法为在线策略（on-policy）算法，即必须使用当前策略 $\pi_{\theta}$ 采样得到的数据来计算梯度。直观理解一下策略梯度这个公式，可以发现在每一个状态下，梯度的修改是让策略更多地去采样到带来比较高 $Q$ 值的动作，更少地去采样到带来比较低 $Q$ 值的动作，如图 9-1 所示。

<div align="center">
    <img src="./image/9-1.png" width = 80%>
    <center>图 9-1 策略梯度示意图</center>
</div>

- 在计算策略梯度的公式中，我们需要用到 $Q^{\pi_{\theta}(s,a)}$，可以用多种方式对它进行估计。接下来要介绍的 REINFORCE 算法便是采用了蒙特卡洛方法来估计 $Q^{\pi_{\theta}}(s,a)$，对于一个有限步数的环境来说，REINFORCE 算法中的策略梯度为：

$$
\nabla_\theta J(\theta)=\mathbb{E}_{\pi_\theta}\left[\sum_{t=0}^T\left(\sum_{t^{\prime}=t}^T\gamma^{t^{\prime}-t}r_{t^{\prime}}\right)\nabla_\theta\log\pi_\theta(a_t|s_t)\right]
$$

- 其中，$T$ 是和环境交互的最大步数。例如，在车杆环境中，$T = 200$。

## 9.3 REINFORCE
- REINFORCE 算法的具体算法流程如下：
    - 初始化策略参数 $\theta$
    - for 序列 $e = 1 \to E$ do:
        - 用当前策略 $\pi_{\theta}$ 采样轨迹 $\{ s_1, a_1, r_1, s_2, a_2, r_2, ...s_T, a_T, r_T\}$
        - 计算当前轨迹每个时刻 t 往后的回报 $\sum_{t'=t}^T\gamma^{t'-t}r_{t'}$，记为 $\psi_{t}$
        - 对 $\theta$ 进行更新，$\theta=\theta+\alpha\sum_t^T\psi_t\nabla_\theta\log\pi_\theta(a_t|s_t)$
    - end for
- 这便是 REINFORCE 算法的全部流程了。接下来让我们来用代码来实现它，看看效果如何吧！

## 9.4 REINFORCE 代码实践
- 我们在车杆环境中进行 REINFORCE 算法的实验。

In [ ]:
import gym
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import rl_utils

- 首先定义策略网络 `PolicyNet`，其输入是某个状态，输出则是该状态下的动作概率分布，这里采用在离散动作空间上的 `softmax()` 函数来实现一个可学习的 **多项分布**（multinomial distribution）。

In [ ]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

- 再定义我们的 REINFORCE 算法。在函数 `take_action()` 函数中，我们通过动作概率分布对离散的动作进行采样。在更新过程中，我们按照算法将损失函数写为策略回报的负数，即 $-\sum_t\psi_t\nabla_\theta\log\pi_\theta(a_t|s_t)$，对 $\theta$ 求导后就可以通过梯度下降来更新策略。

In [ ]:
class REINFORCE:
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma,
                 device):
        self.policy_net = PolicyNet(state_dim, hidden_dim,
                                    action_dim).to(device)
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(),
                                          lr=learning_rate)  # 使用Adam优化器
        self.gamma = gamma  # 折扣因子
        self.device = device

    def take_action(self, state):  # 根据动作概率分布随机采样
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        probs = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action.item()

    def update(self, transition_dict):
        reward_list = transition_dict['rewards']
        state_list = transition_dict['states']
        action_list = transition_dict['actions']

        G = 0
        self.optimizer.zero_grad()
        for i in reversed(range(len(reward_list))):  # 从最后一步算起
            reward = reward_list[i]
            state = torch.tensor([state_list[i]],
                                 dtype=torch.float).to(self.device)
            action = torch.tensor([action_list[i]]).view(-1, 1).to(self.device)
            log_prob = torch.log(self.policy_net(state).gather(1, action))
            G = self.gamma * G + reward
            loss = -log_prob * G  # 每一步的损失函数
            loss.backward()  # 反向传播计算梯度
        self.optimizer.step()  # 梯度下降

- 定义好策略，我们就可以开始实验了，看看 REINFORCE 算法在车杆环境上表现如何吧！

In [ ]:
learning_rate = 1e-3
num_episodes = 1000
hidden_dim = 128
gamma = 0.98
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

env_name = "CartPole-v0"
env = gym.make(env_name)
env.seed(0)
torch.manual_seed(0)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = REINFORCE(state_dim, hidden_dim, action_dim, learning_rate, gamma,
                  device)

return_list = []
for i in range(10):
    with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            transition_dict = {
                'states': [],
                'actions': [],
                'next_states': [],
                'rewards': [],
                'dones': []
            }
            state = env.reset()
            done = False
            while not done:
                action = agent.take_action(state)
                next_state, reward, done, _ = env.step(action)
                transition_dict['states'].append(state)
                transition_dict['actions'].append(action)
                transition_dict['next_states'].append(next_state)
                transition_dict['rewards'].append(reward)
                transition_dict['dones'].append(done)
                state = next_state
                episode_return += reward
            return_list.append(episode_return)
            agent.update(transition_dict)
            if (i_episode + 1) % 10 == 0:
                pbar.set_postfix({
                    'episode':
                    '%d' % (num_episodes / 10 * i + i_episode + 1),
                    'return':
                    '%.3f' % np.mean(return_list[-10:])
                })
            pbar.update(1)

- 在 CartPole-v0 环境中，满分就是 200 分，我们发现 REINFORCE 算法效果很好，可以达到 200 分。接下来我们绘制训练过程中每一条轨迹的回报变化图。由于回报抖动比较大，往往会进行平滑处理。

In [ ]:
episodes_list = list(range(len(return_list)))
plt.plot(episodes_list, return_list)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('REINFORCE on {}'.format(env_name))
plt.show()

mv_return = rl_utils.moving_average(return_list, 9)
plt.plot(episodes_list, mv_return)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('REINFORCE on {}'.format(env_name))
plt.show()

- 可以看到，随着收集到的轨迹越来越多，REINFORCE 算法有效地学习到了最优策略。不过，相比于前面的 DQN 算法，REINFORCE 算法使用了更多的序列，这是因为 REINFORCE 算法是一个在线策略算法，之前收集到的轨迹数据不会被再次利用。此外，REINFORCE 算法的性能也有一定程度的波动，这主要是因为每条采样轨迹的回报值波动比较大，这也是 REINFORCE 算法主要的不足。

## 9.5 小结
- REINFORCE 算法是策略梯度乃至强化学习的典型代表，智能体根据当前策略直接和环境交互，通过采样得到的轨迹数据直接计算出策略参数的梯度，进而更新当前策略，使其向最大化策略期望回报的目标靠近。这种学习方式是典型的从交互中学习，并且其优化的目标（即策略期望回报）正是最终所使用策略的性能，这比基于价值的强化学习算法的优化目标（一般是时序差分误差的最小化）要更加直接。 REINFORCE 算法理论上是能保证局部最优的，它实际上是借助蒙特卡洛方法采样轨迹来估计动作价值，这种做法的一大优点是可以得到无偏的梯度。但是，正是因为使用了蒙特卡洛方法，REINFORCE 算法的梯度估计的方差很大，可能会造成一定程度上的不稳定，这也是第 10 章将介绍的 Actor-Critic 算法要解决的问题。

## 9.6 扩展阅读：策略梯度证明
- 策略梯度定理是强化学习中的重要理论。本节我们来证明 $\nabla_\theta J(\theta)\propto\sum_{s\in S}\nu^{\pi_\theta}(s)\sum_{a\in A}Q^{\pi_\theta}(s,a)\nabla_\theta\pi_\theta(a|s)$

- 先从状态价值函数的推导开始：

$$
\begin{aligned}
\nabla_{\theta}V^{\pi_{\theta}}(s) & =\nabla_\theta(\sum_{a\in A}\pi_\theta(a|s)Q^{\pi_\theta}(s,a)) \\
 & =\sum_{a\in A}(\nabla_\theta\pi_\theta(a|s)Q^{\pi_\theta}(s,a)+\pi_\theta(a|s)\nabla_\theta Q^{\pi_\theta}(s,a)) \\
 & =\sum_{a\in A}(\nabla_\theta\pi_\theta(a|s)Q^{\pi_\theta}(s,a)+\pi_\theta(a|s)\nabla_\theta\sum_{s^{\prime},r}p(s^{\prime},r|s,a)(r+\gamma V^{\pi_\theta}(s^{\prime})) \\
 & =\sum_{a\in A}(\nabla_\theta\pi_\theta(a|s)Q^{\pi_\theta}(s,a)+\gamma\pi_\theta(a|s)\sum_{s^{\prime},r}p(s^{\prime},r|s,a)\nabla_\theta V^{\pi_\theta}(s^{\prime})) \\
 & =\sum_{a\in A}(\nabla_{\theta}\pi_{\theta}(a|s)Q^{\pi_{\theta}}(s,a)+\gamma\pi_{\theta}(a|s)\sum_{s^{\prime}}p(s^{\prime}|s,a)\nabla_{\theta}V^{\pi_{\theta}}(s^{\prime}))
\end{aligned}
$$

- 为了简化表示，我们让 $\phi(s)=\sum_{a\in A}\nabla_\theta\pi_\theta(a|s)Q^{\pi_\theta}(s,a)$，定义 $d^{\pi_{\theta}}(s \to x,k)$ 为策略 $\pi$ 从状态 $s$ 出发 $k$ 步后到达状态 $x$ 的概率。我们继续推导：

$$
\begin{aligned}
\nabla_{\theta}V^{\pi_{\theta}}(s) & =\phi(s)+\gamma\sum_a\pi_\theta(a|s)\sum_{s^{\prime}}P(s^{\prime}|s,a)\nabla_\theta V^{\pi_\theta}(s^{\prime}) \\
 & =\phi(s)+\gamma\sum_a\sum_{s^{\prime}}\pi_\theta(a|s)P(s^{\prime}|s,a)\nabla_\theta V^{\pi_\theta}(s^{\prime}) \\
 & =\phi(s)+\gamma\sum_{s^{\prime}}d^{\pi_\theta}(s\to s^{\prime},1)\nabla_\theta V^{\pi_\theta}(s^{\prime}) \\
 & =\phi(s)+\gamma\sum_{s^{\prime}}d^{\pi_\theta}(s\to s^{\prime},1)[\phi(s^{\prime})+\gamma\sum_{s^{\prime\prime}}d^{\pi_\theta}(s^{\prime}\to s^{\prime\prime},1)\nabla_\theta V^{\pi_\theta}(s^{\prime\prime})] \\
 & =\phi(s)+\gamma\sum_{s^{\prime}}d^{\pi_{\theta}}(s\to s^{\prime},1)\phi(s^{\prime})+\gamma^{2}\sum_{s^{\prime\prime}}d^{\pi_{\theta}}(s\to s^{\prime\prime},2)\nabla_{\theta}V^{\pi_{\theta}}(s^{\prime\prime}) \\
 & =\phi(s)+\gamma\sum_{s^{\prime}}d^{\pi_{\theta}}(s\to s^{\prime},1)\phi(s^{\prime})+\gamma^{2}\sum_{s^{\prime\prime}}d^{\pi_{\theta}}(s^{\prime}\to s^{\prime\prime},2)\phi(s^{\prime\prime})+\gamma^{3}\sum_{s^{\prime\prime}}d^{\pi_{\theta}}(s\to s^{\prime\prime\prime},3)\nabla_{\theta}V^{\pi_{\theta}}(s^{\prime\prime\prime}) \\
 & =\ldots \\
 & =\sum_{x\in S}\sum_{k=0}^\infty\gamma^kd^{\pi_\theta}(s\to x,k)\phi(x)
\end{aligned}
$$

- 定义 $\eta(s)=\mathbb{E}_{s_0}[\sum_{k=0}^\infty\gamma^kd^{\pi_\theta}(s_0\to s,k)]$。至此，回到目标函数：
$$
\begin{aligned}
\nabla_{\theta}J(\theta) & =\nabla_\theta\mathbb{E}_{s_0}[V^{\pi_\theta}(s_0)] \\
 & =\sum_s\mathbb{E}_{s_0}\left[\sum_{k=0}^\infty\gamma^kd^{\pi_\theta}(s_0\to s,k)\right]\phi(s) \\
 & =\sum_s\eta(s)\phi(s) \\
 & =\left(\sum_s\eta(s)\right)\sum_s\frac{\eta(s)}{\sum_s\eta(s)}\phi(s) \\
 & \propto\sum_s\frac{\eta(s)}{\sum_s\eta(s)}\phi(s) \\
 & =\sum_s\nu^{\pi_\theta}(s)\sum_aQ^{\pi_\theta}(s,a)\nabla_\theta\pi_\theta(a|s)
\end{aligned}
$$

- 证明完毕！